In [1]:
# HuggingFace 로그인 - 실제 사용 시 본인의 토큰으로 교체하세요
# !huggingface-cli login --token YOUR_HF_TOKEN
print("⚠️ 보안상의 이유로 실제 토큰이 제거되었습니다. 본인의 HuggingFace 토큰을 사용하세요.")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
The token `llm` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `llm`


## 데이터셋 불러오기

In [2]:
import jsonlines
from datasets import Dataset

# JSONLines 파일 경로
jsonl_path = "/workspace/dataset/train_data.jsonl"

# JSONLines 파일을 읽어서 데이터셋 생성
indataset = []
with jsonlines.open(jsonl_path) as f:
    for lineno, line in enumerate(f.iter(), start=1):
        try:
            # 템플릿에 따라 instruction과 response 형식을 맞춰서 저장
            template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
            indataset.append(template.format(**line))
        except Exception as e:
            # 문제가 있는 줄과 오류를 출력하여 확인
            print(f"Error at line {lineno}: {e}")

# 데이터셋 확인
print('데이터셋 생성 완료')

# Hugging Face Dataset으로 변환
indataset = Dataset.from_dict({'text': indataset})

# 데이터셋 정보 확인
print(indataset)


데이터셋 생성 완료
Dataset({
    features: ['text'],
    num_rows: 4000
})


## Gemma 7B 모델 파인튜닝

In [3]:
import torch

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

In [4]:
# Hugging Face Basic Mode
# base_model = "google/gemma-7b"
# base_model = "google/gemma-7b-it"
# base_model = "google/codegemma-7b"
base_model = "google/codegemma-7b-it"

#### 4비트 양자화 QLoRA 설정

In [5]:
if torch.cuda.get_device_capability()[0] >= 8:
    print("CUDA device capability is 8 or higher. Installing flash-attn.")
    !pip install -qqq flash-attn
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
    print("flash-attn installed successfully. Using flash_attention_2 with torch.bfloat16.")
else:
    print("CUDA device capability is below 8. Using eager mode.")
    attn_implementation = "eager"
    torch_dtype = torch.float16
    print("Using eager implementation with torch.float16.")


# QLoRA config
# quant_config = None

# QLoRA config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=False,
)

CUDA device capability is 8 or higher. Installing flash-attn.
flash-attn installed successfully. Using flash_attention_2 with torch.bfloat16.


#### Gemma 7B 모델 불러오기

In [6]:
# GPU 개수 및 이름 확인

# API 토큰 설정 - 실제 사용 시 환경변수로 설정하세요
# api_token = os.getenv("HUGGINGFACE_TOKEN")
api_token = "YOUR_HF_TOKEN"  # 실제 토큰으로 교체하세요

if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    print(f"Available GPUs: {num_gpus}")
    for i in range(num_gpus):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("No GPU available. Using CPU.")

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    # device_map={"": 0},
    device_map="auto",
    # device_map="balanced"
    token=api_token   
)

model.config.use_cache = False
model.config.pretraining_tp = 1

Available GPUs: 1
GPU 0: NVIDIA H100 NVL


2024-11-12 08:55:33.944071: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731401733.957303   19766 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731401733.960889   19766 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-12 08:55:33.976156: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
model

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 3072, padding_idx=0)
    (layers): ModuleList(
      (0-27): 28 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear4bit(in_features=3072, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=3072, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=3072, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=3072, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear4bit(in_features=3072, out_features=24576, bias=False)
          (up_proj): Linear4bit(in_features=3072, out_features=24576, bias=False)
          (down_proj): Linear4bit(in_features=24576, out_features=3072, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm((3072,), eps=1e-06)
        (post_attention_layernorm):

## 토크나이저 불러오기

In [8]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True, token=api_token) # huggingfase hub에서 제공하는 사용자 정의된 토크나이저 코드를 신뢰하고 실행
tokenizer.pad_token = tokenizer.eos_token # 시퀀스 길이를 맞추기 위해 문장 끝에 eos_token 사용
tokenizer.padding_side = "right" # 패딩 토큰을 시퀀스 어느 쪽에 할지

In [9]:
from peft import get_peft_model, LoraConfig

peft_params = LoraConfig(
    lora_alpha=16, # 스케일링 파라미터
    lora_dropout=0.1,
    r=4, # 저차원 공간의 크기
    bias="none", # 편향 학습 유무
    task_type="CAUSAL_LM", # GPT 계열 모델
    target_modules=["q_proj", "v_proj", "k_proj", "out_proj", "fc_in", "fc_out"] # query와 projection 레이어를 의미 ("q_proj", "v_proj", "k_proj", "out_proj", "fc_in", "fc_out")
)

model = get_peft_model(model, peft_params)
model.print_trainable_parameters()

trainable params: 2,408,448 || all params: 8,540,089,344 || trainable%: 0.0282


## Fine-tuning 파라미터

In [2]:
import os
from trl import SFTConfig, SFTTrainer
from transformers import TrainingArguments

model_path = os.path.join("/mnt/ssd/1/hub", "models--iljoodeephub-Bllossom--llama-3.2-Korean-Bllossom-3B_bf16_lr16_qlr4")

# SFTConfig 설정
sft_config = SFTConfig(
    output_dir=model_path,
    dataset_text_field="text",  # 텍스트 필드 지정
    max_seq_length=512,        # 최대 시퀀스 길이 설정
    packing=False               # 입력 시퀀스의 길이에 따라 배치 그룹화 설정
)

# TrainingArguments 설정
training_params = TrainingArguments(
    output_dir=sft_config.output_dir,
    num_train_epochs=25,                          # 전체 데이터셋을 몇 번 반복할지 설정하는 epoch
    per_device_train_batch_size=8,               # 각 GPU(또는 CPU)에서 학습할 때 사용하는 배치 크기 설정 
    gradient_accumulation_steps=1,                # 그래디언트 누적 단계 설정 
    optim="paged_adamw_8bit",                     # 옵티마이저 / 최적화 알고리즘으로 Adam2 사용
    save_steps=100000,                            # 몇 step 마다 모델을 저장할지 설정하는 파라미터 
    logging_steps=100000,                         # 학습 중 로그를 기록하는 빈도를 설정
    learning_rate=2e-4,                           # 학습률 설정 
    weight_decay=0.001,                           # 가중치 감쇠 설정 (모델의 가중치를 정규화하여 과적합 방지에 사용)
    fp16=False,                                   # 반정밀도(float16) 연산을 사용할지 여부 
    bf16=True,                                    # bfloat16 연산을 사용할지 여부 
    max_grad_norm=0.3,                            # gradient clipping을 위한 최대 노름 값 설정 (gradient 폭주 방지)
    max_steps=-1,                                 # 학습 종료 시 최대 스텝 수 설정 / -1은 스텝 수로 학습을 제한하지 않음 
    warmup_ratio=0.03,                            # 학습률 워밍업의 비율을 설정 / 학습 초기 단계에서 점진적으로 학습률을 증가시켜 안정적인 학습을 유도
    group_by_length=sft_config.packing,           # SFTConfig의 packing 설정을 사용
    lr_scheduler_type="constant",                 # 학습률 스케줄러의 유형 설정 
    report_to="tensorboard"                       # 로그 기록 툴 
)

# 파인 튜닝 트레이너
trainer = SFTTrainer(
    model=model,
    train_dataset=indataset, 
    tokenizer=tokenizer,
    args=training_params,
    dataset_text_field=sft_config.dataset_text_field,  # SFTConfig에서 설정한 텍스트 필드
)

NameError: name 'model' is not defined

In [11]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=50000, training_loss=0.28052732421875, metrics={'train_runtime': 24864.5629, 'train_samples_per_second': 8.044, 'train_steps_per_second': 2.011, 'total_flos': 4.475197204101882e+18, 'train_loss': 0.28052732421875, 'epoch': 50.0})